 # Stream Stats

 This example shows how to clean up a stream of data by removing outliers and
 and averaging the values over time. 

 ## Working path

 Set the working path where the data is stored. For now we'll use the
 provided example data in this current directory.

 But the path could be any where on your computer. For example, if you have a
 folder called "data" in your home directory, you could set the path to:
 `path = "U:\\data\\processing\\Campgain2023_of_aswsome\\data"`

In [ ]:
# all the imports, but we'll go through them one by one as we use them
import os
import matplotlib.pyplot as plt
from particula.data import loader_interface, settings_generator, stream_stats
from particula.data.tests.example_data.get_example_data import get_data_folder

# set the parent directory of the data folder
path = get_data_folder()
print('Path to data folder:')
print(path.rsplit('particula')[-1])

# Load the data

For this example we'll use the provided example data. But you can change the
path to any folder on your computer. We then can used the settings generator to
load the data.

In [ ]:
# This method uses the settings_generator module to generate the settings.
settings = settings_generator.for_general_1d_load(
    relative_data_folder='CPC_3010_data',
    filename_regex='*.csv',
    file_min_size_bytes=10,
    data_checks={
        "characters": [10, 100],
        "char_counts": {",": 4},
        "skip_rows": 0,
        "skip_end": 0,
    },
    data_column=[1, 2],
    data_header=['CPC_count[#/sec]', 'Temperature[degC]'],
    time_column=[0],
    time_format='epoch',
    delimiter=',',
    time_shift_seconds=0,
    timezone_identifier='UTC',
)

# now call the loader interface
data_stream = loader_interface.load_files_interface(
    path=path,
    settings=settings,
)

In [ ]:
# print data stream summary
print('Stream:')
print(data_stream)

In [ ]:
# plot the data
fig, ax = plt.subplots()
ax.plot(data_stream.datetime64,
        data_stream.data[0, :],  # data_stream.data is a 2d array, so we need
                                 # to specify which column we want to plot
        label=data_stream.header[0],
        linestyle="none",
        marker=".",)
plt.xticks(rotation=45)
ax.set_xlabel("Time (UTC)")
ax.set_ylabel(data_stream.header[0])
plt.show()
fig.tight_layout()

# Average the data

Now that we have the data loaded, we can average the data over time. We'll use
the 'particula.data.stream_stats' module to do this. The module has a function
called 'averaged_std' that will take stream object and return a new stream
object with the averaged data and the standard deviation of the data.

In [ ]:
stream_averaged = stream_stats.average_std(
    stream=data_stream,
    average_interval=600,
)
stream_averaged.standard_deviation.shape

# Plot the averaged data

In [ ]:
fig, ax = plt.subplots()
ax.plot(stream_averaged.datetime64,
        stream_averaged.data[0, :],
        label=stream_averaged.header[0],
        marker=".",)
plt.xticks(rotation=45)
ax.set_xlabel("Time (UTC)")
ax.set_ylabel(stream_averaged.header[0])
plt.show()
fig.tight_layout()

## Clean up the data

Now we may see some outliers in the data. We can use the 'particula.data.stream_stats'
module to remove the outliers. The module has a function called 'filtering'
that will take stream object and return a new stream object with the outliers
removed.

In [ ]:
stream_filtered = stream_stats.filtering(
    stream=data_stream,
    top=250000,
    drop=True,
)
fig, ax = plt.subplots()
ax.plot(stream_filtered.datetime64,
        stream_filtered.data[0, :],
        label=stream_filtered.header[0],
        marker=".",)
plt.xticks(rotation=45)
ax.set_xlabel("Time (UTC)")
ax.set_ylabel(stream_filtered.header[0])
plt.show()
fig.tight_layout()


## Summary

This example shows how to clean up a stream of data by removing outliers and
and averaging the values over time.


In [ ]:
help(stream_stats)